In [7]:
# import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import shutil
# import cv2
from scipy import ndimage
# from skimage.transform import resize
from joblib import Parallel, delayed
import tensorflow as tf
from sklearn.model_selection import KFold
# from tqdm.auto import tqdm

In [8]:
model_names = os.listdir("/home/statsgeneral/gayara/Tasselnet/Filtered_90/CV_models_F90")

In [9]:
# how many models?
len(model_names)

5

In [10]:
model_names[1]

'model_2.h5'

In [11]:
loaded_models = []
for model in model_names:
    model_ld = tf.keras.models.load_model(r"/home/statsgeneral/gayara/Tasselnet/Filtered_90/CV_models_F90" + "/" + model)
    loaded_models.append(model_ld)

In [12]:
len(loaded_models)

5

In [13]:
# names of the test files
numpy_folder = "/work/statsgeneral/gayara/Filtered_90/np_test_files"
test_files = os.listdir(numpy_folder)

In [14]:
# how many test files
len(test_files)

65

In [15]:
# Function to get the predictions given model and image

numpy_folder = "/work/statsgeneral/gayara/Filtered_90/np_test_files"
v_stack_folder = "/work/statsgeneral/gayara/Filtered_90/test_files_stack"

def prediction_on_test_data(model, selected_file, stride = 8, kernel_size = 32):
#     load the cnn model
    
# load the image data file
    load_image = np.load(numpy_folder + "/"+ selected_file)
    
    # get the image height
    img_height = load_image.shape[0]
    # get the image weight
    img_width = load_image.shape[1]

    all_test_sub_windows = np.load(v_stack_folder + "/"+ selected_file)

    # now, to get the predictions, pass the sub windows
    test_image_prediction = model.predict(all_test_sub_windows, batch_size = 100)
    
    # density map
    Density_map = np.zeros((img_height, img_width))

    # counts map
    counts_map = np.zeros((img_height, img_width))
    
    # now, for every window, we will keep adding the values together and also add the counts
    counter = 0
#     need a counter to move into each predicted value in the pred values list
    for ii in range(0, img_height, stride):
        for jj in range(0, img_width, stride):
#         operations for density map
#             get the window of interest
            new_window = Density_map[ii:ii + kernel_size,jj:jj+kernel_size]
#     fill each with the value c_k
            counts_window = np.full((new_window.shape[0], new_window.shape[1]), test_image_prediction[counter])
#     get the shapes of this new window
            cw_height = counts_window.shape[0]
            cw_width = counts_window.shape[1]
#         Do c_k/r_2
            counts_window_new = counts_window/(cw_height*cw_width)
#     This is the value in the window now
            value_window = counts_window_new
#     place the values in the corrsponding area of the density map
            Density_map[ii:ii + kernel_size,jj:jj+kernel_size] = new_window + value_window

#         Let's now focus on capturing the counts of the windows
            new_window_c = counts_map[ii:ii + kernel_size,jj:jj+kernel_size]
#     get the counts area
            count = np.ones((new_window_c.shape[0], new_window_c.shape[1]))
#     keep adding the counts to reflect the addition of densities
            counts_map[ii:ii + kernel_size,jj:jj+kernel_size] = new_window_c + count
#     increase the counter
            counter = counter + 1
            
#         get the normalized count
    normalized_counts = np.divide(Density_map, counts_map)
    
#     entire count on the test set
    pred_on_test = np.sum(normalized_counts)
    
#     return the predicted value
    return(pred_on_test, normalized_counts, selected_file)


In [16]:
# get the preds and normalized counts

all_preds_loop = []
for mdl in loaded_models:
    preds_sub = []
    for file in test_files:
        results = prediction_on_test_data(mdl, file)
        preds_sub.append(results)
    all_preds_loop.append(preds_sub)

34/40 [========================>.....] - ETA: 0s  

2022-11-21 22:03:55.917300: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-11-21 22:03:55.917788: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


55/55 [==============================] - 0s 2ms/step


In [17]:
len(preds_sub)

65

In [18]:
# Extract the preds, image names, and density maps
all_dense_maps = []
all_preds = []
all_im_names = []
for i in all_preds_loop:
    dense_maps = []
    preds = []
    names = []
    for j in i:
        pred = j[0]
        preds.append(pred)
        name = j[2]
        names.append(name)
        dense_map = j[1]
        dense_maps.append(dense_map)
    all_dense_maps.append(dense_maps)
    all_preds.append(preds)
    all_im_names.append(names)
    


In [19]:
all_dfs = []
for i in range(0, len(all_dense_maps)):
    list_1 = all_im_names[i]
    list_2 = all_preds[i]
    df = pd.DataFrame(list(zip(list_1, list_2)), columns = ["Image_name", "Predicted_count"])
#     remove the image name
    all_dfs.append(df)

In [20]:
# all_dfs

In [21]:
try_df = pd.concat(all_dfs, axis = 1)

In [22]:
try_df_no_im_name = try_df.drop(columns = "Image_name")

In [23]:
try_df_im_name = try_df[["Image_name"]]

In [24]:
try_df_im_name = pd.DataFrame(try_df_im_name.iloc[:,0])

In [25]:
all_preds = pd.concat((try_df_im_name, try_df_no_im_name),axis = 1)

In [26]:
all_preds.head()

,Image_name,Predicted_count,Predicted_count,Predicted_count,Predicted_count,Predicted_count
0,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,44.999617,44.832856,43.509650,47.364557,43.303209
1,190903_djifc6310s_Sept3_SW_row_7_range_6_rep_6...,61.502814,62.524575,59.427361,64.084734,61.813218
2,190903_djifc6310s_Sept3_SE_row_5_range_3_rep_2...,27.828884,28.341484,26.455592,28.587456,27.740400
3,190820_djifc6310s_Aug20_SW_row_22_range_5_rep_...,45.881977,44.558216,44.998663,47.022796,43.881925
4,190823_djifc6310_Aug23_NW_row_39_range_5_rep_7...,14.363523,13.840903,13.344874,15.075551,13.756458


In [27]:
# add increments to the predicted counts
col_names = []
counter = 0
for i in all_preds.columns:
    name = i + '_' + str(counter)
    counter = counter + 1
    col_names.append(name)

In [28]:
# replace the old col names with new col names
all_preds.columns = col_names

In [29]:
all_preds.head()

,Image_name_0,Predicted_count_1,Predicted_count_2,Predicted_count_3,Predicted_count_4,Predicted_count_5
0,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,44.999617,44.832856,43.509650,47.364557,43.303209
1,190903_djifc6310s_Sept3_SW_row_7_range_6_rep_6...,61.502814,62.524575,59.427361,64.084734,61.813218
2,190903_djifc6310s_Sept3_SE_row_5_range_3_rep_2...,27.828884,28.341484,26.455592,28.587456,27.740400
3,190820_djifc6310s_Aug20_SW_row_22_range_5_rep_...,45.881977,44.558216,44.998663,47.022796,43.881925
4,190823_djifc6310_Aug23_NW_row_39_range_5_rep_7...,14.363523,13.840903,13.344874,15.075551,13.756458


In [30]:
# Import the true counts csv file
true_counts = pd.read_csv("/home/statsgeneral/gayara/Tasselnet/Filtered_90/true_counts_test_only_sample20.csv")

In [31]:
true_counts.head()

,Image_name,True_count
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,53
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,21
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,19
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,33
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,50


In [32]:
true_count_col = true_counts['True_count']

In [33]:
true_counts.iloc[0,0]

'190812_djifc6310s_Aug12_NW_row_32_range_6_rep_6_isref_1_90.png'

In [34]:
all_preds.iloc[0,0]

'190814_djifc6310s_Aug14_NE_row_33_range_2_rep_5_90.npy'

In [54]:
# notice that the names are different with respect to the position. We need to therefore merge using a key

In [35]:
true_counts['Image_name'] = true_counts['Image_name'].apply(lambda x: x.split('.')[0]+'.npy')

# # all_names_only = []
# for i in true_counts['Image_name']:
#     name = i.split('.')[0]
#     name_new = name + '.npy'
    
# ### SPLIT COLUMNS INTO A LIST 
# df['information'] = df['information'].apply(lambda x: x.replace("|", "").split("\n"))

# ### EXPLODE THE COLUMN
true_counts.explode('Image_name')

,Image_name,True_count
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,53
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,21
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,19
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,33
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,50
...,...,...
60,190905_djifc6310s_Sept5_SE_row_12_range_6_rep_...,32
61,190905_djifc6310s_Sept5_SW_row_11_range_2_rep_...,24
62,190905_djifc6310s_Sept5_SW_row_31_range_2_rep_...,43
63,190905_djifc6310s_Sept5_SW_row_32_range_5_rep_...,26


In [74]:
# rename the image name column in all_preds

# df = df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'})

In [36]:
all_preds = all_preds.rename(columns = {"Image_name_0":"Image_name"})

In [37]:
all_preds.head()

,Image_name,Predicted_count_1,Predicted_count_2,Predicted_count_3,Predicted_count_4,Predicted_count_5
0,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,44.999617,44.832856,43.509650,47.364557,43.303209
1,190903_djifc6310s_Sept3_SW_row_7_range_6_rep_6...,61.502814,62.524575,59.427361,64.084734,61.813218
2,190903_djifc6310s_Sept3_SE_row_5_range_3_rep_2...,27.828884,28.341484,26.455592,28.587456,27.740400
3,190820_djifc6310s_Aug20_SW_row_22_range_5_rep_...,45.881977,44.558216,44.998663,47.022796,43.881925
4,190823_djifc6310_Aug23_NW_row_39_range_5_rep_7...,14.363523,13.840903,13.344874,15.075551,13.756458


In [38]:
true_counts.iloc[0,0]

'190812_djifc6310s_Aug12_NW_row_32_range_6_rep_6_isref_1_90.npy'

In [39]:
all_preds.iloc[1,0]

'190903_djifc6310s_Sept3_SW_row_7_range_6_rep_6_isref_1_90.npy'

In [40]:
final_trial_data = true_counts.merge(all_preds, how = 'right', on = 'Image_name')

In [41]:
final_trial_data.iloc[1,0]

'190903_djifc6310s_Sept3_SW_row_7_range_6_rep_6_isref_1_90.npy'

In [42]:
final_trial_data.head()

,Image_name,True_count,Predicted_count_1,Predicted_count_2,Predicted_count_3,Predicted_count_4,Predicted_count_5
0,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,50,44.999617,44.832856,43.509650,47.364557,43.303209
1,190903_djifc6310s_Sept3_SW_row_7_range_6_rep_6...,53,61.502814,62.524575,59.427361,64.084734,61.813218
2,190903_djifc6310s_Sept3_SE_row_5_range_3_rep_2...,35,27.828884,28.341484,26.455592,28.587456,27.740400
3,190820_djifc6310s_Aug20_SW_row_22_range_5_rep_...,27,45.881977,44.558216,44.998663,47.022796,43.881925
4,190823_djifc6310_Aug23_NW_row_39_range_5_rep_7...,8,14.363523,13.840903,13.344874,15.075551,13.756458


In [43]:
# get the substracted values from each column
preds_only_df = final_trial_data.drop(['Image_name','True_count'], axis = 1)

In [44]:
preds_only_df

,Predicted_count_1,Predicted_count_2,Predicted_count_3,Predicted_count_4,Predicted_count_5
0,44.999617,44.832856,43.509650,47.364557,43.303209
1,61.502814,62.524575,59.427361,64.084734,61.813218
2,27.828884,28.341484,26.455592,28.587456,27.740400
3,45.881977,44.558216,44.998663,47.022796,43.881925
4,14.363523,13.840903,13.344874,15.075551,13.756458
...,...,...,...,...,...
60,14.724014,14.892021,13.813080,15.306008,14.726681
61,46.079107,45.625405,43.758280,46.895475,45.015620
62,35.065501,34.325877,33.833517,36.042452,33.523139
63,28.912711,29.590006,27.221017,30.634902,29.479856


In [45]:
true_count_col = final_trial_data['True_count']

In [46]:
subtracted_cols = abs(preds_only_df.sub(true_count_col, axis = 0))

In [47]:
subtracted_cols.mean()

Predicted_count_1    8.738208
Predicted_count_2    8.526849
Predicted_count_3    8.308142
Predicted_count_4    9.038493
Predicted_count_5    8.327667
dtype: float64

In [48]:
mean_all = subtracted_cols.mean().mean()
mean_all

8.58787194715568

In [49]:
std_all = subtracted_cols.mean().std()

In [50]:
lower_bnd = mean_all - 1.96*(std_all/np.sqrt(65))

In [51]:
lower_bnd

8.513401887445458

In [52]:
upper_bnd = mean_all + 1.96*(std_all/np.sqrt(65))

In [53]:
upper_bnd

8.662342006865902

In [54]:
# Save the dataframe with preds for future use
final_trial_data.head()

,Image_name,True_count,Predicted_count_1,Predicted_count_2,Predicted_count_3,Predicted_count_4,Predicted_count_5
0,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,50,44.999617,44.832856,43.509650,47.364557,43.303209
1,190903_djifc6310s_Sept3_SW_row_7_range_6_rep_6...,53,61.502814,62.524575,59.427361,64.084734,61.813218
2,190903_djifc6310s_Sept3_SE_row_5_range_3_rep_2...,35,27.828884,28.341484,26.455592,28.587456,27.740400
3,190820_djifc6310s_Aug20_SW_row_22_range_5_rep_...,27,45.881977,44.558216,44.998663,47.022796,43.881925
4,190823_djifc6310_Aug23_NW_row_39_range_5_rep_7...,8,14.363523,13.840903,13.344874,15.075551,13.756458


In [55]:
final_trial_data.to_csv("/home/statsgeneral/gayara/Tasselnet/Filtered_90/all_preds_CV_all_data.csv")

In [56]:
final_trial_data

,Image_name,True_count,Predicted_count_1,Predicted_count_2,Predicted_count_3,Predicted_count_4,Predicted_count_5
0,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,50,44.999617,44.832856,43.509650,47.364557,43.303209
1,190903_djifc6310s_Sept3_SW_row_7_range_6_rep_6...,53,61.502814,62.524575,59.427361,64.084734,61.813218
2,190903_djifc6310s_Sept3_SE_row_5_range_3_rep_2...,35,27.828884,28.341484,26.455592,28.587456,27.740400
3,190820_djifc6310s_Aug20_SW_row_22_range_5_rep_...,27,45.881977,44.558216,44.998663,47.022796,43.881925
4,190823_djifc6310_Aug23_NW_row_39_range_5_rep_7...,8,14.363523,13.840903,13.344874,15.075551,13.756458
...,...,...,...,...,...,...,...
60,190816_djifc6310s_Aug16_NW_row_22_range_2_rep_...,37,14.724014,14.892021,13.813080,15.306008,14.726681
61,190823_djifc6310_Aug23_SE_row_23_range_1_rep_6...,44,46.079107,45.625405,43.758280,46.895475,45.015620
62,190820_djifc6310s_Aug20_SW_row_22_range_6_rep_...,34,35.065501,34.325877,33.833517,36.042452,33.523139
63,190816_djifc6310s_Aug16_SW_row_22_range_6_rep_...,38,28.912711,29.590006,27.221017,30.634902,29.479856
